<a href="https://colab.research.google.com/github/syahputra21/Tugas5_Deplearning_Yolov8_DeepSort/blob/main/Tugas5_DeteksiPenghitungan_Orang_Yolov8%2BDeepSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics deep-sort-realtime opencv-python-headless ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
  

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
from IPython.display import HTML, display
from base64 import b64encode
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Initialize YOLOv8 model
model = YOLO("yolov8n.pt")  # Use yolov8s.pt for better accuracy

# Initialize DeepSORT tracker
tracker = DeepSort(
    max_age=30,  # How long to keep lost tracks
    n_init=3,    # Number of detections before track is confirmed
    max_iou_distance=0.7,  # Maximum IoU distance for matching
    embedder="mobilenet",  # Feature extractor
    half=True,   # Use half precision for faster processing
    bgr=True     # Expect BGR images (OpenCV default)
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 323MB/s]


In [10]:
print("📤 Please upload your video file...")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"✅ Uploaded video: {video_path}")

📤 Please upload your video file...


Saving Video testing.mp4 to Video testing (1).mp4
✅ Uploaded video: Video testing (1).mp4


In [11]:
# Open video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError("❌ Failed to open video!")

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"📏 Video Info: {width}x{height} resolution")
print(f"🎞️ {fps} FPS, {frame_count} frames total")
print(f"⏱️ Estimated duration: {frame_count/fps:.2f} seconds")

# Prepare output
output_path = 'output_counting.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

📏 Video Info: 596x336 resolution
🎞️ 29.97002997002997 FPS, 900 frames total
⏱️ Estimated duration: 30.03 seconds


In [12]:
counted_ids = set()
total_count = 0
frame_num = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_num += 1
    if frame_num % 10 == 0:
        print(f"⏳ Processing frame {frame_num}/{frame_count}", end='\r')

    # Convert BGR to RGB for YOLO
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect people
    results = model(frame_rgb, classes=[0], verbose=False)[0]  # Class 0 = person
    detections = []

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()
        detections.append(([x1, y1, x2-x1, y2-y1], conf, 'person'))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame_rgb)

    # Visualize results
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())

        # Draw bounding box
        cv2.rectangle(frame, (l, t), (r, b), (0, 200, 255), 2)
        cv2.putText(frame, f'ID:{track_id}', (l, t-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Count new people
        if track_id not in counted_ids:
            counted_ids.add(track_id)
            total_count += 1

    # Display counter
    cv2.putText(frame, f'Total: {total_count}', (20, 50),
               cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    # Write frame
    out.write(frame)

cap.release()
out.release()
print(f"\n✅ Processing complete! Total people counted: {total_count}")

⏳ Processing frame 900/900
✅ Processing complete! Total people counted: 60


In [13]:
# Verify the size of the output video
file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"📁 Output video size: {file_size_mb:.2f} MB")

if file_size_mb < 0.1:  # Check for abnormally small file
    print("⚠️ Caution: The output video is unusually small. Something might be wrong.")
else:
    print("✔️ Output video appears to be valid")

📁 Output video size: 7.53 MB
✔️ Output video appears to be valid


In [14]:
def display_video(file_path):
    # Re-encode video to ensure compatibility
    !ffmpeg -y -i {file_path} -vcodec libx264 output_final.mp4 -hide_banner -loglevel error

    # Load and embed the video in notebook
    with open("output_final.mp4", "rb") as vid:
        encoded = b64encode(vid.read()).decode()
    video_src = f"data:video/mp4;base64,{encoded}"

    display(HTML(f'''
    <div style="margin: 10px 0;">
      <video width="800" controls>
        <source src="{video_src}" type="video/mp4">
      </video>
      <p>Total people detected: {total_count}</p>
    </div>
    '''))

display_video(output_path)

In [15]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>